# Imports

In [84]:
import pandas as pd
from datetime import datetime

# Parameters
The following cell contains variables that are used to drive execution and improve reusability with future data sets. This ensures the process is repeatable for future datasets (aiding in fulfilling steps 5-7 of the project requirements).

These parameters can be modified, which will impact the data pipeline without requiring coding changes, at least not significant ones. Every dataset is different, so future datasets may require code changes to further generalize the processing.

In [85]:
# File Location
strFilePath = "../Data/"
strFileNameRoot = "Project_1_Data_File"
strFileExtension = ".csv"

# Columns to treat as dates when importing
lstDateColNames = ["Date"]

# Columns to treat as doubles when importing
lstDoubleColNames = ["Transaction Amount"]

# TODO: Add more data types for future (more varied) input data files - Might be better to use a dictionary of "Column":"Datatype"

# Columns to standardize values - this will be used to cleanse categorical data, i.e. handle typos. For example, when looking at
# New York county data, we could have entries for "ST. LAWRENCE" and "SAINT LAWRENCE" county, and we want "SAINT LAWRENCE" to be correct.
# We could add: the following to the dictionary: "County": {"ST. LAWRENCE":"SAINT LAWRENCE"}

# Format:
#  dictColMappingData = {
#   "Column Name": {"Old Value":"New Value"}
# }
dictColMappingData = {
    "Payment Method": {"CC":"Credit Card",
                       "PP":"PayPal"}
}

# Calculated column definitions. These will be applied AFTER data cleansing, once dfClean (the clean dataset) is created
dictCalculatedColData = {
    # Example of a lookup, i.e. the value in new column "transaction fee" is based on the values in the "payment method" column
    "Transaction Fee Pct": # Column to be created
        {
            "Column Type": "Lookup",
            "Data Type": "double[pyarrow]",
            "Column Spec":
            {
                "Payment Method": # Column containing the categories we need to look up
                {
                    "Credit Card":0.03, # If "Payment Method" == "Credit Card", the new column value will be 0.03
                    "PayPal":0.02,
                    "DEFAULT_VALUE":0.00 # Fallback to 0.00 if the payment method does not incur a transaction fee
                }
            }
        },
    # Example of a column calculated based on other column values
    "Total Transaction Fee": 
    {
        "Column Type": "Function",
        "Data Type": "double[pyarrow]",
        "Column Spec": lambda row: round(row["Transaction Amount"] * row["Transaction Fee Pct"], 2)
    },

    # Another calculated column - notice that since we are going to end up applying these in order,
    # column definitions can reference earlier columns.
    "Total Transaction Amt": 
    {
        "Column Type": "Function",
        "Data Type": "double[pyarrow]",
        "Column Spec": lambda row: round(row["Transaction Amount"] + row["Total Transaction Fee"], 2)
    }
}

### Calculated variables used throughout, based on the parameters

In [86]:
# Append the file path and file name into a single variable, ensuring we handle the case where the user did not finish the path with a slash
strFullFilePath = strFilePath + ("" if strFilePath[-1:] == "/" else "/") + strFileNameRoot + strFileExtension
strCleanFilePath = strFilePath + ("" if strFilePath[-1:] == "/" else "/") + strFileNameRoot + "_Clean" + strFileExtension
strErrorFilePath = strFilePath + ("" if strFilePath[-1:] == "/" else "/") + strFileNameRoot + "_Errors" + strFileExtension
print(strFullFilePath, "\n", strCleanFilePath, "\n", strErrorFilePath)

../Data/Project_1_Data_File.csv 
 ../Data/Project_1_Data_File_Clean.csv 
 ../Data/Project_1_Data_File_Errors.csv


# Step 1
Data Ingestion
- Choose the correct source for the provided sample dataset
- Write code to fetch and load the raw data into a data structure

In [87]:
dfImport = pd.read_csv(strFullFilePath, dtype_backend="pyarrow")

# Step 2
Data Assessment and Exploration
- Examine the dataset to identify missing values, outliers, and data quality issues
- To understand the data, generate summary statistics (mean, median, mode, etc)

In [88]:
# Ensure we can look at the full dataset
#pd.set_option('display.max_rows', 10)
print(dfImport.dtypes)
display(dfImport)
dfImport.describe(include="all")

Name                  string[pyarrow]
Email                 string[pyarrow]
Date                  string[pyarrow]
Transaction Amount    string[pyarrow]
Payment Method        string[pyarrow]
Address               string[pyarrow]
Product Name          string[pyarrow]
dtype: object


,Name,Email,Date,Transaction Amount,Payment Method,Address,Product Name
0,Amy Frye,fusce.diam@hotmail.net,11/25/2022,17.52,Credit Card,"373-5327 Vulputate, Av.",Wiberg Cure
1,Travis Tyler,phasellus.libero.mauris@outlook.net,5/17/2023,16.54,Cash,"P.O. Box 699, 2987 Lacus. St.",Lettuce - Sea / Sea Asparagus
2,Linda Herrera,pede@hotmail.com,8/25/2017,16.6,Credit Card,Ap #930-3501 Nulla. St.,Mangostein
3,Cody Dotson,ac@google.ca,12/15/2018,13.48,CC,"P.O. Box 713, 5080 Lacinia Rd.",Bulgar
4,Aretha Wilkerson,sit@icloud.net,4/24/2024,14.39,Cash,Ap #839-4960 Ornare Rd.,Cocoa Powder - Dutched
...,...,...,...,...,...,...,...
995,Eliana Thornton,nullam.suscipit.est@outlook.edu,7/31/2018,12.24,Credit Card,5365 Augue St.,Veal - Osso Bucco
996,Drake Luna,gravida.mauris@google.com,2/19/2022,15.18,Cash,642-5311 Nunc Road,Muffin Hinge Container 6
997,Hilda Hall,aliquam.erat.volutpat@google.com,4/28/2022,20.12,Credit Card,652-9803 Velit Av.,Chutney Sauce
998,September Townsend,vitae.sodales.at@hotmail.org,6/7/2017,15.74,Cash,"918-6561 Ultrices, Rd.","Jam - Blackberry, 20 Ml Jar"


,Name,Email,Date,Transaction Amount,Payment Method,Address,Product Name
count,1000,1000,995,995,1000,1000,1000
unique,1000,974,858,608,5,1000,829
top,Amy Frye,Not Provided,4/4/2022,13.84,Credit Card,"373-5327 Vulputate, Av.",Arizona - Green Tea
freq,1,18,4,6,699,1,4


At this point, our ability to analyze the data is hampered by the fact that the bad data values are being brought in as strings. We'll proceed to step 3 (cleansing), and complete those steps, then return to the mean/median/mode.

# Step 3
Data Cleaning
- Handle missing values: replace or remove them based on the context
- Address outliers or incorrect values
- Standardize data types for consistency
- Clean and format text or categorical data as necessary

### Bad Data Checks
Reviewing the file in Excel found the following:
- 5 rows with blank dates (those rows will need to be excluded from any date-sensitive calculations)
- multiple date formats (standardize these)
- 7 rows with a value of "Missing" or blank in the Transaction Amount column (exclude these rows entirely)

### Reusable functions
The next cell defines reusable functions that can be used for cleaning up columns of a certain type, defined at the top.

In [89]:
# Define reusable functions that can be used on a column.

# Date cleansing function which parses known date formats; returns mm/dd/yyyy, or None if the date can't be parsed.
def clean_date(strInputDate):
    lstValidFormats = ["%m/%d/%Y", "%m-%d-%Y", "%Y-%m-%d"]

    for fmt in lstValidFormats:
        try:
            cleandate = datetime.strptime(strInputDate, fmt)
            # If we were able to parse the date, return the standardized %m/%d/%Y format
            return datetime.strftime(cleandate, "%m/%d/%Y")
        except:
            pass
    
    # If we've made it here, none of the date patterns matched our data
    return None

# Numeric (double) cleansing function - removes any characters that are invalid for casting as a double
def clean_double(strInputDouble):
    try:
        return ''.join(char for char in strInputDouble if char in set("0123456789.eE-+"))
    except:
        return None

# Apply string corrections - strInputValue exists in dictColMappingData, return it; otherwise return strInputValue.
# This ensures that we do NOT need to include ALL values in the dictionary only those we need to change.
def standardize_string_columns(strColName, strInputValue):
    try:
        strReturnValue = dictColMappingData[strColName][strInputValue]
        return strReturnValue
    except:
        return strInputValue

# Apply calculated columns
def calculated_columns(row, calc_col_def):
    # Get the calculation type from the column definition
    calc_col_type = calc_col_def["Column Type"]

    if calc_col_type == "Lookup":
        # Extract the key and value (which is our lookup dictionary) from calc_col_def
        for base_col_name, dictLookup in calc_col_def["Column Spec"].items():
            if row[base_col_name] in dictLookup:
                return dictLookup[row[base_col_name]]
            elif "DEFAULT_VALUE" in dictLookup:
                return dictLookup["DEFAULT_VALUE"]
            else:
                return None

    elif calc_col_type == "Function":
        return calc_col_def["Column Spec"](row)

    # Default - i.e. the calculated column is of a type we have not written code to handle yet
    else:
        return None         

# # Apply calculated columns
# def calculated_columns(row, calc_col_def):
#     # If the calculated column is defined as a lookup, apply it based on the dictionary
#     if isinstance(calc_col_def, dict):
#         # Extract the key and value (which is our lookup dictionary) from calc_col_def - there will only be one item
#         for base_col_name, dictLookup in calc_col_def.items():
#             if row[base_col_name] in dictLookup:
#                 return dictLookup[row[base_col_name]]
#             elif "DEFAULT_VALUE" in dictLookup:
#                 return dictLookup["DEFAULT_VALUE"]
#             else:
#                 return None
    
#     # Now handle cases where the calculated column is defined as a function/calculation
#     elif callable(calc_col_def):
#         return calc_col_def(row)
    
#     # Default - i.e. the calculated column is of a different type, which we have not defined yet at this time
#     else:
#         return None

### Apply the cleansing functions defined above

In [90]:
# Apply the data cleansing functions
for c in lstDateColNames:
    for i in dfImport.index:
        dfImport.loc[i, c] = clean_date(dfImport.loc[i, c])

for c in lstDoubleColNames:
    for i in dfImport.index:
        dfImport.loc[i, c] = clean_double(dfImport.loc[i, c])

for c in dictColMappingData:
    # Only proceed if the column exists in dfImport
    if c in dfImport.columns:
        for i in dfImport.index:
            dfImport.loc[i, c] = standardize_string_columns(c, dfImport.loc[i, c])

### Create the clean dataset

After this cell, dfDirty will contain our error records, and dfClean will contain our "good" records. Since dirty and clean mean different things on different projects, this is not abstracted to configuration settings, so these formulas will need to be adjusted for different datasets.

In [91]:
# Create a dataframe containing records we corrected as much as possible but still need to be scrubbed out due to lingering issues.
# The idea here is the provider of the original file will make corrections and run them back through the pipeline.
dfDirty = dfImport[((dfImport["Date"].isna()) | (dfImport["Transaction Amount"].str.upper()=="MISSING") | (dfImport["Transaction Amount"].isna()) | (dfImport["Transaction Amount"]==""))].copy()

# Create the opposite - a new dataframe that excludes our major data anomalies that we can't correct
dfClean = dfImport[~((dfImport["Date"].isna()) | (dfImport["Transaction Amount"].str.upper()=="MISSING") | (dfImport["Transaction Amount"].isna()) | (dfImport["Transaction Amount"]==""))].copy()

print("Before:\n", dfClean.dtypes)

# Now we can apply the proper datatypes to the dataframe
for d in lstDateColNames:
    dfClean[d] = dfClean[d].astype("date32[pyarrow]")
for d in lstDoubleColNames:
    dfClean[d] = dfClean[d].astype("double[pyarrow]")

print("\nAfter:\n", dfClean.dtypes)

display(dfClean)

Before:
 Name                  string[pyarrow]
Email                 string[pyarrow]
Date                  string[pyarrow]
Transaction Amount    string[pyarrow]
Payment Method        string[pyarrow]
Address               string[pyarrow]
Product Name          string[pyarrow]
dtype: object

After:
 Name                       string[pyarrow]
Email                      string[pyarrow]
Date                  date32[day][pyarrow]
Transaction Amount         double[pyarrow]
Payment Method             string[pyarrow]
Address                    string[pyarrow]
Product Name               string[pyarrow]
dtype: object


,Name,Email,Date,Transaction Amount,Payment Method,Address,Product Name
0,Amy Frye,fusce.diam@hotmail.net,2022-11-25,17.52,Credit Card,"373-5327 Vulputate, Av.",Wiberg Cure
1,Travis Tyler,phasellus.libero.mauris@outlook.net,2023-05-17,16.54,Cash,"P.O. Box 699, 2987 Lacus. St.",Lettuce - Sea / Sea Asparagus
2,Linda Herrera,pede@hotmail.com,2017-08-25,16.6,Credit Card,Ap #930-3501 Nulla. St.,Mangostein
3,Cody Dotson,ac@google.ca,2018-12-15,13.48,Credit Card,"P.O. Box 713, 5080 Lacinia Rd.",Bulgar
4,Aretha Wilkerson,sit@icloud.net,2024-04-24,14.39,Cash,Ap #839-4960 Ornare Rd.,Cocoa Powder - Dutched
...,...,...,...,...,...,...,...
995,Eliana Thornton,nullam.suscipit.est@outlook.edu,2018-07-31,12.24,Credit Card,5365 Augue St.,Veal - Osso Bucco
996,Drake Luna,gravida.mauris@google.com,2022-02-19,15.18,Cash,642-5311 Nunc Road,Muffin Hinge Container 6
997,Hilda Hall,aliquam.erat.volutpat@google.com,2022-04-28,20.12,Credit Card,652-9803 Velit Av.,Chutney Sauce
998,September Townsend,vitae.sodales.at@hotmail.org,2017-06-07,15.74,Cash,"918-6561 Ultrices, Rd.","Jam - Blackberry, 20 Ml Jar"


### Revisit step 2 to compare metrics before and after our data cleansing

In [92]:
dfImport.describe(include="all")

,Name,Email,Date,Transaction Amount,Payment Method,Address,Product Name
count,1000,1000,995,995,1000,1000,1000
unique,1000,974,857,608,3,1000,829
top,Amy Frye,Not Provided,04/04/2022,13.84,Credit Card,"373-5327 Vulputate, Av.",Arizona - Green Tea
freq,1,18,4,6,716,1,4


In [93]:
dfClean.describe(include="all")

,Name,Email,Date,Transaction Amount,Payment Method,Address,Product Name
count,989,989,989,989.0,989,989,989
unique,989,963,NaN,<NA>,3,989,822
top,Amy Frye,Not Provided,NaN,<NA>,Credit Card,"373-5327 Vulputate, Av.",Arizona - Green Tea
freq,1,18,NaN,<NA>,707,1,4
mean,NaN,NaN,2020-03-29,15.308372,NaN,NaN,NaN
min,NaN,NaN,2015-09-04,-12.0,NaN,NaN,NaN
25%,NaN,NaN,2018-02-03,13.56,NaN,NaN,NaN
50%,NaN,NaN,2020-03-30,15.1,NaN,NaN,NaN
75%,NaN,NaN,2022-06-09,16.78,NaN,NaN,NaN
max,NaN,NaN,2024-08-19,79.6,NaN,NaN,NaN


# Step 4
Data Transformation
- Create a derived column. For example, you could create a column for the amount paid as a processing fee, assuming credit cards have a 3% fee, PayPal has a 2% fee, and cash has no fee.

In [94]:
# Utilize dictCalculatedColData to apply our calculated columns
for calc_col_name, calc_col_def in dictCalculatedColData.items():
    print("Creating column ", calc_col_name)
    dfClean[calc_col_name] = dfClean.apply(calculated_columns, args=(calc_col_def,), axis=1)
    print("Applying data type ", calc_col_def["Data Type"], " to column ", calc_col_name)
    dfClean[calc_col_name] = dfClean[calc_col_name].astype(calc_col_def["Data Type"])

Creating column  Transaction Fee Pct
Applying data type  double[pyarrow]  to column  Transaction Fee Pct
Creating column  Total Transaction Fee
Applying data type  double[pyarrow]  to column  Total Transaction Fee
Creating column  Total Transaction Amt
Applying data type  double[pyarrow]  to column  Total Transaction Amt


In [95]:
print(dfClean.dtypes)
display(dfClean)

Name                          string[pyarrow]
Email                         string[pyarrow]
Date                     date32[day][pyarrow]
Transaction Amount            double[pyarrow]
Payment Method                string[pyarrow]
Address                       string[pyarrow]
Product Name                  string[pyarrow]
Transaction Fee Pct           double[pyarrow]
Total Transaction Fee         double[pyarrow]
Total Transaction Amt         double[pyarrow]
dtype: object


,Name,Email,Date,Transaction Amount,Payment Method,Address,Product Name,Transaction Fee Pct,Total Transaction Fee,Total Transaction Amt
0,Amy Frye,fusce.diam@hotmail.net,2022-11-25,17.52,Credit Card,"373-5327 Vulputate, Av.",Wiberg Cure,0.03,0.53,18.05
1,Travis Tyler,phasellus.libero.mauris@outlook.net,2023-05-17,16.54,Cash,"P.O. Box 699, 2987 Lacus. St.",Lettuce - Sea / Sea Asparagus,0.0,0.0,16.54
2,Linda Herrera,pede@hotmail.com,2017-08-25,16.6,Credit Card,Ap #930-3501 Nulla. St.,Mangostein,0.03,0.5,17.1
3,Cody Dotson,ac@google.ca,2018-12-15,13.48,Credit Card,"P.O. Box 713, 5080 Lacinia Rd.",Bulgar,0.03,0.4,13.88
4,Aretha Wilkerson,sit@icloud.net,2024-04-24,14.39,Cash,Ap #839-4960 Ornare Rd.,Cocoa Powder - Dutched,0.0,0.0,14.39
...,...,...,...,...,...,...,...,...,...,...
995,Eliana Thornton,nullam.suscipit.est@outlook.edu,2018-07-31,12.24,Credit Card,5365 Augue St.,Veal - Osso Bucco,0.03,0.37,12.61
996,Drake Luna,gravida.mauris@google.com,2022-02-19,15.18,Cash,642-5311 Nunc Road,Muffin Hinge Container 6,0.0,0.0,15.18
997,Hilda Hall,aliquam.erat.volutpat@google.com,2022-04-28,20.12,Credit Card,652-9803 Velit Av.,Chutney Sauce,0.03,0.6,20.72
998,September Townsend,vitae.sodales.at@hotmail.org,2017-06-07,15.74,Cash,"918-6561 Ultrices, Rd.","Jam - Blackberry, 20 Ml Jar",0.0,0.0,15.74


# Step 5/6
Create the output files

In [96]:
# Error file
dfDirty.to_csv(strErrorFilePath, index=False)
# Clean File
dfClean.to_csv(strCleanFilePath, index=False)